In [5]:
import pandas as pd

df = pd.read_excel('单品数据总表.xlsx')
print(df)

                  单品编码       销售日期  销量(千克)      销售额  批发价格(元/千克)    当日成本价  \
0      102900005115168 2020-10-30   6.427  32.1350        3.00  19.2810   
1      102900005115168 2020-10-31   7.251  36.2550        3.00  21.7530   
2      102900005115168 2020-11-01  13.126  60.3796        3.00  39.3780   
3      102900005115168 2020-12-04   5.972  17.9160        1.60   9.5552   
4      102900005115168 2021-03-03  11.568  57.8400        1.60  18.5088   
...                ...        ...     ...      ...         ...      ...   
46594  106973990980123 2022-07-23   2.000   4.0000        1.67   3.3400   
46595  106973990980123 2022-07-25   7.000  17.5000        1.87  13.0900   
46596  106973990980123 2022-07-26   7.000  12.5000        1.90  13.3000   
46597  106973990980123 2022-07-28  10.000  21.0000        2.02  20.2000   
46598  106973990980123 2022-07-29   2.000   5.0000        2.03   4.0600   

            利润率    当日平均定价 分类名称  
0      0.666667  5.000000  花叶类  
1      0.666667  5.000000  花叶类  


In [6]:
ef = pd.read_excel('分频数各品类回归数据/x,y值.xlsx')
print(ef)


    分类名称       截距      斜率
0    花叶类  253.260 -12.542
1    花菜类   63.621  -2.664
2  水生根茎类   69.628  -3.311
3     茄类   30.149  -0.999
4    辣椒类  106.217  -2.523
5    食用菌  114.208  -5.140


In [7]:
df_selected = df[['单品编码', '销售日期', '销售额', '当日平均定价','分类名称']]
print(df_selected)

                  单品编码       销售日期      销售额    当日平均定价 分类名称
0      102900005115168 2020-10-30  32.1350  5.000000  花叶类
1      102900005115168 2020-10-31  36.2550  5.000000  花叶类
2      102900005115168 2020-11-01  60.3796  4.600000  花叶类
3      102900005115168 2020-12-04  17.9160  3.000000  花叶类
4      102900005115168 2021-03-03  57.8400  5.000000  花叶类
...                ...        ...      ...       ...  ...
46594  106973990980123 2022-07-23   4.0000  2.000000  食用菌
46595  106973990980123 2022-07-25  17.5000  2.500000  食用菌
46596  106973990980123 2022-07-26  12.5000  1.785714  食用菌
46597  106973990980123 2022-07-28  21.0000  2.100000  食用菌
46598  106973990980123 2022-07-29   5.0000  2.500000  食用菌

[46599 rows x 5 columns]


In [8]:

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
grouped_data = df_selected.groupby('单品编码')


result = pd.DataFrame(columns=['单品编码', '截距', '斜率'])



for group_name, group_data in grouped_data:
    # Filter out outliers based on daily average pricing
    group_data = group_data[np.abs(group_data['当日平均定价'] - group_data['当日平均定价'].mean()) <= group_data['当日平均定价'].std()]
    
    # Continue with the rest of the code...

for group_name, group_data in grouped_data:
    ef_row = ef[ef['分类名称'] == group_data['分类名称'].iloc[0]]
    intercept = ef_row['截距'].iloc[0]
    slope = ef_row['斜率'].iloc[0]
    #print(intercept)
    y = group_data['销售额']
    X = group_data[['当日平均定价']]
    
    
    model = LinearRegression()
    model.fit(X, y)
    if(model.coef_>10):
        model.coef_[0]=slope 
        model.intercept_=intercept
    n = len(y)*0.4/(365*3)  # replace with desired weight
    model.intercept_ = intercept*(1-n)+model.intercept_*n
    model.coef_ = np.array([slope*(1-n)+model.coef_[0]*n])
    
    # Print the adjusted coefficients
    print(f'Adjusted coefficients for group {group_name}: {model.coef_[0]}')
    # Print the adjusted intercept
    print(f'Adjusted intercept for group {group_name}: {model.intercept_}')
    # # Print the adjusted R-squared score
    # print(f'Adjusted R-squared score for group {group_name}: {model.score(X, y)}')
    result = result.append({'单品编码': group_name, '截距': model.intercept_, '斜率': model.coef_[0]}, ignore_index=True)

result.to_excel('单品线性结果.xlsx', index=False)


Adjusted coefficients for group 102900005115168: -11.82131208878873
Adjusted intercept for group 102900005115168: 244.0738564487454
Adjusted coefficients for group 102900005115199: -11.839984318769192
Adjusted intercept for group 102900005115199: 242.76163197281062
Adjusted coefficients for group 102900005115250: -5.779804622866351
Adjusted intercept for group 102900005115250: 149.76138036124115
Adjusted coefficients for group 102900005115625: -12.337855872362237
Adjusted intercept for group 102900005115625: 249.1671506627123
Adjusted coefficients for group 102900005115748: -12.17119116931001
Adjusted intercept for group 102900005115748: 246.02443681636825
Adjusted coefficients for group 102900005115762: -9.573191635951677
Adjusted intercept for group 102900005115762: 203.4539719261287
Adjusted coefficients for group 102900005115779: -6.522332135937223
Adjusted intercept for group 102900005115779: 206.49258600423462
Adjusted coefficients for group 102900005115786: -9.798806106863983
Ad